In [1]:
!pip install aiogram==1.2.3

In [2]:
!pip install vedis

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

class ContentLoss(nn.Module):
        def __init__(self, target,):
            super(ContentLoss, self).__init__()
            self.target = target.detach()#это константа. Убираем ее из дерева вычеслений
            self.loss = F.mse_loss(self.target, self.target )#to initialize with something

        def forward(self, input):
            self.loss = F.mse_loss(input, self.target)
            return input
        
class StyleLoss(nn.Module):
        def __init__(self, target_feature):
            super(StyleLoss, self).__init__()
            self.target = gram_matrix(target_feature).detach()
            self.loss = F.mse_loss(self.target, self.target)# to initialize with something
            
        def gram_matrix(input):
            batch_size , h, w, f_map_num = input.size()  # batch size(=1)
            # b=number of feature maps
            # (h,w)=dimensions of a feature map (N=h*w)
    
            features = input.view(batch_size * h, w * f_map_num)  # resise F_XL into \hat F_XL
    
            G = torch.mm(features, features.t())  # compute the gram product
    
            # we 'normalize' the values of the gram matrix
            # by dividing by the number of element in each feature maps.
            return G.div(batch_size * h * w * f_map_num)

        def forward(self, input):
            G = gram_matrix(input)
            self.loss = F.mse_loss(G, self.target)
            return input
        
        
class Normalization(nn.Module):
        def __init__(self, mean, std):
            super(Normalization, self).__init__()
            # .view the mean and std to make them [C x 1 x 1] so that they can
            # directly work with image Tensor of shape [B x C x H x W].
            # B is batch size. C is number of channels. H is height and W is width.
            self.mean = torch.tensor(mean).view(-1, 1, 1)
            self.std = torch.tensor(std).view(-1, 1, 1)

        def forward(self, img):
            # normalize img
            return (img - self.mean) / self.std

def gram_matrix(input):
        batch_size , h, w, f_map_num = input.size()  # batch size(=1)
        # b=number of feature maps
        # (h,w)=dimensions of a feature map (N=h*w)

        features = input.view(batch_size * h, w * f_map_num)  # resise F_XL into \hat F_XL

        G = torch.mm(features, features.t())  # compute the gram product

        # we 'normalize' the values of the gram matrix
        # by dividing by the number of element in each feature maps.
        return G.div(batch_size * h * w * f_map_num)

In [16]:
from PIL import Image


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


import torchvision.transforms as transforms
import torchvision.models as models

import copy


        
class StyleTransferModel:
    def __init__(self,device, cnn,normalization_mean, normalization_std,
                 style_img, content_img,
                 content_layers=['conv_4'],
                 style_layers=['conv_1', 'conv_2', 'conv_3', 'conv_4', 'conv_5']):
        
        cnn = copy.deepcopy(cnn)
        self.device = device
        style_img = self._preprocesses(style_img).to(self.device, torch.float)
        content_img = self._preprocesses(content_img).to(self.device, torch.float)

        # normalization module
        normalization = Normalization(normalization_mean, normalization_std).to(device)

        # just in order to have an iterable access to or list of content/syle
        # losses
        content_losses = []
        style_losses = []

        # assuming that cnn is a nn.Sequential, so we make a new nn.Sequential
        # to put in modules that are supposed to be activated sequentially
        model = nn.Sequential(normalization)

        i = 0  # increment every time we see a conv
        for layer in cnn.children():
            if isinstance(layer, nn.Conv2d):
                i += 1
                name = 'conv_{}'.format(i)
            elif isinstance(layer, nn.ReLU):
                name = 'relu_{}'.format(i)
                layer = nn.ReLU(inplace=False)
            elif isinstance(layer, nn.MaxPool2d):
                name = 'pool_{}'.format(i)
            elif isinstance(layer, nn.BatchNorm2d):
                name = 'bn_{}'.format(i)
            else:
                raise RuntimeError('Unrecognized layer: {}'.format(layer.__class__.__name__))

            model.add_module(name, layer)

            if name in content_layers:
                # add content loss:
                target = model(content_img).detach()
                content_loss = ContentLoss(target)
                model.add_module("content_loss_{}".format(i), content_loss)
                content_losses.append(content_loss)

            if name in style_layers:
                # add style loss:
                target_feature = model(style_img).detach()
                style_loss = StyleLoss(target_feature)
                model.add_module("style_loss_{}".format(i), style_loss)
                style_losses.append(style_loss)

        # now we trim off the layers after the last content and style losses
        for i in range(len(model) - 1, -1, -1):
            if isinstance(model[i], ContentLoss) or isinstance(model[i], StyleLoss):
                break

        self.model = model[:(i + 1)]
        self.style_losses = style_losses
        self.content_losses = content_losses
    
        
    def __call__(self, input_img, num_steps=500, style_weight=100000, content_weight = 1):
        
        input_img = self._preprocesses(input_img).to(self.device,torch.float)
        optimizer = optim.LBFGS([input_img.requires_grad_()]) 

        print('Optimizing..')
        run = [0]
        while run[0] <= num_steps:

            def closure():
                # correct the values 
                # это для того, чтобы значения тензора картинки не выходили за пределы [0;1]
                input_img.data.clamp_(0, 1)

                optimizer.zero_grad()

                self.model(input_img)

                style_score = 0
                content_score = 0

                for sl in self.style_losses:
                    style_score += sl.loss
                for cl in self.content_losses:
                    content_score += cl.loss
                
                #взвешивание ощибки
                style_score *= style_weight
                content_score *= content_weight

                loss = style_score + content_score
                loss.backward()

                run[0] += 1
                if run[0] % 50 == 0:
                    print("run {}:".format(run))
                    print('Style Loss : {:4f} Content Loss: {:4f}'.format(
                        style_score.item(), content_score.item()))
                    print()

                return style_score + content_score

            optimizer.step(closure)

        # a last correction...
        input_img.data.clamp_(0, 1)

        return input_img
    
    def _preprocesses(self, img):
        imsize = 128
        loader = transforms.Compose([
            transforms.Resize(imsize),  # нормируем размер изображения
            transforms.CenterCrop(imsize),
            transforms.ToTensor()])
        img = loader(img).unsqueeze(0)
        return img

In [5]:
from enum import Enum
db_file = "database.vdb"
class States(Enum):
    S_START = "0"  # Начало нового диалога
    S_STYLE = "1"
    S_CONTENT = "2"

In [6]:
from vedis import Vedis

# Пытаемся узнать из базы «состояние» пользователя
def get_current_state(user_id):
    with Vedis(db_file) as db:
        try:
            return db[user_id].decode() # Если используете Vedis версии ниже, чем 0.7.1, то .decode() НЕ НУЖЕН
        except KeyError:  # Если такого ключа почему-то не оказалось
            return States.S_START.value  # значение по умолчанию - начало диалога

def set_state(user_id, value):
    with Vedis(db_file) as db:
        try:
            db[user_id] = value
            return True
        except:
            return False

In [7]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [8]:
from aiogram import Bot, types
from aiogram.dispatcher import Dispatcher

from aiogram.utils import executor
from aiogram.utils.helper import Helper, HelperMode, ListItem
from PIL import Image
from io import BytesIO
from aiogram.types import ReplyKeyboardRemove, ReplyKeyboardMarkup, KeyboardButton

TOKEN = '1239115079:AAHB2lI3iB2K23fYVC9iZTtRp1XFCp0IWUE'

In [9]:
button_hi = KeyboardButton('Давай!')
greet_kb = ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True).add(button_hi)

In [ ]:
bot = Bot(token=TOKEN)
dp = Dispatcher(bot)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cnn = models.vgg19(pretrained=True).features.to(device).eval()
cnn_normalization_mean = torch.tensor([0.485, 0.456, 0.406]).to(device)
cnn_normalization_std = torch.tensor([0.229, 0.224, 0.225]).to(device)
image1 = False
image2 = False

@dp.message_handler(commands=['start'])
async def process_start_command(message: types.Message):
  set_state(message.from_user.id, str(States.S_START.value))
  await message.reply("Привет!\nЯ умею переносить стиль на фотографии, давай поиграем!", 
                      reply_markup=greet_kb)
  
@dp.message_handler(func=lambda message: get_current_state(message.chat.id) == States.S_START.value, text = "Давай!")
async def process_start_command(message: types.Message):
  set_state(message.from_user.id, str(States.S_STYLE.value))
  await message.reply("Отправь мне картинку стиля")
    
@dp.message_handler(commands=['help'])
async def process_help_command(message: types.Message):
    await message.reply("Напиши мне что-нибудь, и я отправлю этот текст тебе в ответ!")

@dp.message_handler(func=lambda message: get_current_state(message.chat.id) == States.S_START.value, commands=['start_style_transfer'])    
async def process_start_style_command(message: types.Message):
    set_state(message.from_user.id, States.S_STYLE.value)
    await message.reply("Отправь мне картинку стиля")

@dp.message_handler(func=lambda message: get_current_state(message.chat.id) == States.S_STYLE.value, content_types=types.ContentType.PHOTO)    
async def get_style(message: types.Message):
    set_state(message.from_user.id, States.S_CONTENT.value)
    global image1
    image1 = await bot.download_file_by_id(message.photo[-1].file_id)
    image1 = Image.open(image1)
    # await message.photo[-1].download('data/%s/style.jpg' %
    #                               (message.from_user.id))
    await message.reply('Отлично, со стилем определились.\n Теперь отправь мне изображение, стиль которого ты хочешь изменить')

@dp.message_handler(func=lambda message: get_current_state(message.chat.id) == States.S_CONTENT.value, content_types=types.ContentType.PHOTO)    
async def get_style(message: types.Message):
    image2 = await bot.download_file_by_id(message.photo[-1].file_id)
    image2 = Image.open(image2)
    image3 = image2
    await message.reply('Окейсики, теперь подожди немного')
    model = StyleTransferModel(device,cnn,cnn_normalization_mean, cnn_normalization_std, image1, image2)
    img = image3
    unloader = transforms.ToPILImage()
    img = model(img)
    img2 = unloader(img.squeeze(0))
    bio = BytesIO()
    bio.name = 'image.jpeg'
    img2.save(bio, 'JPEG')
    bio.seek(0)
    await bot.send_photo(message.from_user.id,photo = bio)
    set_state(message.from_user.id, States.S_START.value)

async def set_dir(message, type_path):
    path = 'data/%s/%s' % (message.from_user.id, type_path)
    if not os.path.exists(path):
        os.makedirs(path)
    
@dp.message_handler()
async def echo_message(msg: types.Message):
    await bot.send_message(msg.from_user.id, msg.text)
    
if __name__ == '__main__':
    executor.start_polling(dp)

Start bot with long-polling.
Cause exception while getting updates.
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/aiogram/bot/api.py", line 199, in request
    async with session.post(url, data=data, **kwargs) as response:
  File "/usr/local/lib/python3.6/dist-packages/aiohttp/client.py", line 1012, in __aenter__
    self._resp = await self._coro
  File "/usr/local/lib/python3.6/dist-packages/aiohttp/client.py", line 504, in _request
    await resp.start(conn)
  File "/usr/local/lib/python3.6/dist-packages/aiohttp/client_reqrep.py", line 847, in start
    message, payload = await self._protocol.read()  # type: ignore  # noqa
  File "/usr/local/lib/python3.6/dist-packages/aiohttp/streams.py", line 591, in read
    await self._waiter
aiohttp.client_exceptions.ServerDisconnectedError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/aiogram/dispatcher/_

Optimizing..
run [50]:
Style Loss : 9.749947 Content Loss: 20.823372

run [100]:
Style Loss : 4.677236 Content Loss: 16.698208

run [150]:
Style Loss : 3.441935 Content Loss: 15.300850

run [200]:
Style Loss : 2.906644 Content Loss: 14.645202

run [250]:
Style Loss : 2.678251 Content Loss: 14.281075

run [300]:
Style Loss : 2.532545 Content Loss: 14.061394

run [350]:
Style Loss : 2.456171 Content Loss: 13.923008

run [400]:
Style Loss : 2.410214 Content Loss: 13.824737

run [450]:
Style Loss : 2.376648 Content Loss: 13.754292

run [500]:
Style Loss : 2.363316 Content Loss: 13.698855



Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f5979c52c50>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x7f5979cca4c0>, 7030.907647803)]']
connector: <aiohttp.connector.TCPConnector object at 0x7f597c24d400>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f597c24ce80>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x7f5979cca388>, 7072.345706009)]']
connector: <aiohttp.connector.TCPConnector object at 0x7f5979c52b70>


Optimizing..
run [50]:
Style Loss : 16.983826 Content Loss: 32.090790

run [100]:
Style Loss : 7.763161 Content Loss: 27.084064

run [150]:
Style Loss : 4.637348 Content Loss: 25.090958

run [200]:
Style Loss : 3.403345 Content Loss: 24.020432

run [250]:
Style Loss : 2.903208 Content Loss: 23.380596

run [300]:
Style Loss : 2.638815 Content Loss: 22.974009

run [350]:
Style Loss : 2.487755 Content Loss: 22.687504

run [400]:
Style Loss : 2.390603 Content Loss: 22.512508

run [450]:
Style Loss : 2.338969 Content Loss: 22.374727

run [500]:
Style Loss : 2.307351 Content Loss: 22.279539



Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f5979c73f98>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x7f5979cca8d0>, 7432.0287757)]']
connector: <aiohttp.connector.TCPConnector object at 0x7f597c242320>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f597c24d2e8>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x7f5979caace0>, 7439.985611289)]']
connector: <aiohttp.connector.TCPConnector object at 0x7f597c249128>


Optimizing..
run [50]:
Style Loss : 9.749947 Content Loss: 20.823372

run [100]:
Style Loss : 4.677236 Content Loss: 16.698208

run [150]:
Style Loss : 3.441935 Content Loss: 15.300850

run [200]:
Style Loss : 2.906644 Content Loss: 14.645202

run [250]:
Style Loss : 2.678251 Content Loss: 14.281075

run [300]:
Style Loss : 2.532545 Content Loss: 14.061394

run [350]:
Style Loss : 2.456171 Content Loss: 13.923008

run [400]:
Style Loss : 2.410214 Content Loss: 13.824737

run [450]:
Style Loss : 2.376648 Content Loss: 13.754292

run [500]:
Style Loss : 2.363316 Content Loss: 13.698855



Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f5979c78b38>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x7f5979caa458>, 8670.735512639)]']
connector: <aiohttp.connector.TCPConnector object at 0x7f5979c78ba8>
Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7f597c24d748>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x7f5979cca4c0>, 8993.665806061)]']
connector: <aiohttp.connector.TCPConnector object at 0x7f5979c789b0>


Optimizing..
run [50]:
Style Loss : 26.613243 Content Loss: 15.056137

run [100]:
Style Loss : 18.513912 Content Loss: 14.578157

run [150]:
Style Loss : 10.449574 Content Loss: 15.282975

run [200]:
Style Loss : 5.130457 Content Loss: 14.767145

run [250]:
Style Loss : 3.976939 Content Loss: 14.112865

run [300]:
Style Loss : 3.655138 Content Loss: 13.847616

run [350]:
Style Loss : 3.449576 Content Loss: 13.695054

run [400]:
Style Loss : 3.298433 Content Loss: 13.590355

run [450]:
Style Loss : 3.191171 Content Loss: 13.511128

run [500]:
Style Loss : 3.112575 Content Loss: 13.446594

